# What is a python decorator?

- Python’s decorators allow you to __extend and modify the behavior__ of a callable (functions, methods, and classes) __without permanently modifying the callable__ itself.




- TWO THINGS TO KEEP IN MIND:
    - __Functions are objects__ ==> they can be assigned to variables and passed to and returned from other functions.
    - __Functions can be defined inside other functions__ ==> and a child function can capture the parent function’s local state

- They “decorate” or “wrap” another function and let you execute code before and after the wrapped function runs.


- Decorators allow you to define reusable building blocks that can change or extend the behavior of other functions.

In [1]:
def null_decorator(func):
    return func

- null_decorator is a callable (it’s a function), it takes another callable as its input, and it returns the same input callable without modifying it.

In [2]:
def greet():
    return 'Hello!'

greet_decorated = null_decorator(greet)

greet_decorated()

'Hello!'

- Instead of explicitly calling null_decorator on greet and then reassigning the greet variable, you can use Python’s @ syntax for decorating a function in one step

In [3]:
@null_decorator
def greet():
    return 'Hello!'

greet()

'Hello!'

- let’s write another decorator that actually does something and modifies the behavior of the decorated function.

In [4]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

In [5]:
@uppercase
def greet():
    return 'Hello!'

greet()

'HELLO!'

# Applying Multiple Decorators to a Single Function

In [6]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper

In [7]:
@strong
@emphasis
def greet():
    return 'Hello!'

In [8]:
greet()

'<strong><em>Hello!</em></strong>'

- __THE DECORATORS ARE APPLIED IN BOTTOM TO TOP ORDER__
- WHICH IS SAME AS:

```python

decorated_greet = strong(emphasis(greet))

```

# Decorating Functions That Accept Arguments

- This is where Python’s [*args and \*\*kwargs](https://www.youtube.com/watch?v=WcTXxX3vYgY) feature for dealing with variable numbers of arguments comes in handy.

```python
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper
```

__There are two notable things going on with this decorator:__

- It uses the * and ** operators in the wrapper closure definition to collect all positional and keyword arguments and stores them in variables (args and kwargs).
- The wrapper closure then forwards the collected arguments to the original input function using the * and ** “argument unpacking” operators.

In [13]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() '
              f'with args: {args}, kwargs: {kwargs}')

        original_result = func(*args, **kwargs)

        print(f'TRACE: {func.__name__}() '
              f'returned {original_result!r}')

        return original_result
    return wrapper

In [14]:
@trace
def say(name, line):
    return f'{name}: {line}'

In [15]:
say('Jane', 'Hello, World')

TRACE: calling say() with args: ('Jane', 'Hello, World'), kwargs: {}
TRACE: say() returned 'Jane: Hello, World'


'Jane: Hello, World'

# How to Write “Debuggable” Decorators

- When you use a decorator, really what you’re doing is __replacing one function with another.__

- One downside of this process is that __it “hides” some of the metadata__ attached to the original (undecorated) function.

In [16]:
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

decorated_greet = uppercase(greet)

In [17]:
greet.__name__

'greet'

In [18]:
greet.__doc__

'Return a friendly greeting.'

In [19]:
decorated_greet.__name__

'wrapper'

In [21]:
print(decorated_greet.__doc__)

None


- __You can use functools.wraps in your own decorators to copy over the lost metadata from the undecorated function to the decorator closure.__

In [22]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

In [23]:
@uppercase
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

In [24]:
greet.__name__

'greet'

In [25]:
greet.__doc__

'Return a friendly greeting.'